In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import time
import torch
import torchvision
import csgm
import dcgan
import skimage
import anchor_image
import image_utils
import importlib
import time
import pickle

In [2]:
importlib.reload(image_utils)

<module 'image_utils' from '/home/kurtis/Desktop/anchor/ImageAnchors/image_utils.py'>

In [3]:
samples = {0: [3],
        1: [2],
        2: [1],
        3: [18],
        4: [4],
        5: [8],
        6: [11],
        7: [0],
        8: [61],
        9: [7]}

In [4]:
mnist_test = torchvision.datasets.MNIST('./', train=False, transform=lambda x: np.array(x)/255.,
                                   target_transform=None, download=True)

In [5]:
for x in samples:
    for c in samples[x]:
        print("NUM SEGMENTS", np.max(image_utils.create_segments(mnist_test[c][0])) + 1)

15 4.000000000000001
NUM SEGMENTS 15
15 3.18125
NUM SEGMENTS 15
15 4.000000000000001
NUM SEGMENTS 15
15 3.8000000000000007
NUM SEGMENTS 15
15 3.828125000000001
NUM SEGMENTS 15
15 3.712500000000001
NUM SEGMENTS 15
15 3.8000000000000007
NUM SEGMENTS 15
15 3.5000000000000004
NUM SEGMENTS 15
15 3.7000000000000006
NUM SEGMENTS 15
15 3.3500000000000005
NUM SEGMENTS 15


## Setting up

In [6]:
parameters = {
    'dummy': {'dataset': mnist_test, 'dummys': mnist_test},
    'batch': {'G': lambda x: dcgan.load_generator(eval=True), 'dataset': mnist_test, 'batch_norm': False},
    'batchBN': {'G': lambda x: dcgan.load_generator(eval=False),'dataset': mnist_test, 'batch_norm': True}
}

In [7]:
net = dcgan.Net()
net.load_state_dict(torch.load('mnist_cnn.pt'))

In [8]:
def predict_fn(images):
    try:
        images = torch.from_numpy(images).type(torch.FloatTensor)
    except:
        pass
    return net(images.view(images.shape[0],1,28,28)).data.numpy()

## Collect data Function

In [9]:
times = {
    key: {i: [] for i in range(10)
         }  for key in parameters}

precisions = {key:{i: [] for i in range(10)}
                 for key in parameters
             }

coverages = {key:{i: [] for i in range(10)}
                 for key in parameters
             }

In [10]:
def collectData(parameter, times, precisions, n_trials = 1):
#     n_trials = 1
    currTime = times[parameter]
    currPrecision = precisions[parameter]
    currCoverage = coverages[parameter]
    for c in samples:
        sample = samples[c][0]
        for i in range(n_trials):
            image = mnist_test[sample][0]
            explainer = anchor_image.AnchorImageMNIST([],'yeet', **parameters[parameter])
            start = time.process_time()
            coverage, segments, exp = explainer.explain_instance(image,predict_fn,verbose=True)
            end = time.process_time()
            currTime[c].append(end-start)
            currPrecision[c].append(exp)
            currCoverage[c].append(coverage)
            with open(parameter+'_times.pck','wb') as f:
                pickle.dump(currTime,f)
            with open(parameter+'_precisions.pck','wb') as f:
                pickle.dump(currPrecision,f)
    

## Collect data

In [11]:
parameter = 'batchBN'

In [ ]:
collectData(parameter,times,precisions,n_trials = 3)

15 4.000000000000001
True pred 0
Taking coverage data...
Checkpoint 1
Making tuples...
Best: 11 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 0 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 10 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 1 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 9 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 2 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 8 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 3 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 6 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 4 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 5 (mean:1.0000000000, n: 1, lb:0.0000) Worst: 7 (mean:0.0000, n: 1, ub:1.0000) B = 1.00
Best: 11 (mean:0.9207920792, n: 101, lb:0.6917) Worst: 12 (mean:0.0000, n: 1, ub:1.0000) B = 0.31
Best: 11 (mean:0.9402985075, n: 201, lb:0.8008) Worst: 13 (mean:0.0000, n: 1, ub:1.0000) B = 0.20
Best: 11 (mean:0.9401993355, n: 301, lb:0.8319) Worst: 14 (mean:0.0000, n: 1, ub:1.0000) B = 0.17
Best of size  1 :
11 0.9501246882793017 0.92330823533

In [2]:
with open('batchBN_precisions.pck', 'rb') as f:
    res = pickle.load(f)

In [3]:
res

{0: [[(11, '', 0.9329529243937232, [], 0), (7, '', 1.0, [], 0)],
  [(11, '', 0.9317803660565723, [], 0), (7, '', 1.0, [], 0)],
  [(11, '', 0.946843853820598, [], 0), (5, '', 1.0, [], 0)]],
 1: [[], [], []],
 2: [[(0, '', 0.8419301164725458, [], 0), (7, '', 1.0, [], 0)],
  [(0, '', 0.8211788211788211, [], 0), (7, '', 1.0, [], 0)],
  [(0, '', 0.8031383737517832, [], 0), (7, '', 1.0, [], 0)]],
 3: [[],
  [(5, '', 0.3932905067808708, [], 0),
   (11, '', 0.9402985074626866, [], 0),
   (8, '', 1.0, [], 0)],
  []],
 4: [[(8, '', 0.7066167290886392, [], 0), (0, '', 1.0, [], 0)],
  [(8, '', 0.7162837162837162, [], 0),
   (5, '', 0.9401993355481728, [], 0),
   (0, '', 1.0, [], 0)],
  [(8, '', 0.6907730673316709, [], 0), (0, '', 1.0, [], 0)]],
 5: [[(3, '', 0.48951048951048953, [], 0), (6, '', 0.9603960396039604, [], 0)],
  [(3, '', 0.5014985014985015, [], 0), (6, '', 0.9867109634551495, [], 0)],
  [(3, '', 0.48501362397820164, [], 0), (6, '', 0.9900332225913622, [], 0)]],
 6: [[(0, '', 0.6833095